<a href="https://colab.research.google.com/github/nvnsudharsan/download_google_embeddings/blob/main/download_google_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install earthengine-api

In [ ]:

import ee
import numpy as np
import os
import argparse
from tqdm import tqdm
import json
from datetime import datetime

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project='ee-my-naveensudharsan')

In [ ]:
bbox = {
    'lon_min': -98.4,
    'lon_max': -96.9,
    'lat_min': 29.7,
    'lat_max': 30.9
}


In [ ]:
# Create Earth Engine geometry
region = ee.Geometry.Rectangle([
    bbox['lon_min'],
    bbox['lat_min'],
    bbox['lon_max'],
    bbox['lat_max']
])

In [ ]:
import numpy as np
from tqdm import tqdm
import json
from datetime import datetime

# Parameters
scale = 800  # meters (matches PRISM resolution)
n_components = 64  # number of embedding dimensions
years = list(range(2017, 2025))

In [ ]:
# Calculate dimensions
lat_range = bbox['lat_max'] - bbox['lat_min']
lon_range = bbox['lon_max'] - bbox['lon_min']
width = int((lon_range * 111000) / scale)
height = int((lat_range * 111000) / scale)

In [ ]:
region = ee.Geometry.Rectangle([bbox['lon_min'], bbox['lat_min'],
                                bbox['lon_max'], bbox['lat_max']])

In [ ]:
file_prefix = 'austin'
folder_name = 'google_embeddings_austin'
export_scale_m = 800  # change to 10 if you want native 10 m
export_crs = 'EPSG:4326'  # EE will reproject on export

In [ ]:
print("Step 4: Downloading embeddings for each year...\n")

region = ee.Geometry.Rectangle([bbox['lon_min'], bbox['lat_min'],
                                bbox['lon_max'], bbox['lat_max']], proj='EPSG:4326', geodesic=False)

collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

downloaded_tasks = []

for year in years:
    print("="*60)
    print(f"YEAR {year}")
    print("="*60)

    start_date = f'{year}-01-01'
    end_date   = f'{year+1}-01-01'  # end is exclusive in EE

    # Filter for the year and your region; take the first matching tile
    year_image = (collection
                  .filterDate(start_date, end_date)
                  .filterBounds(region)
                  .first())

    # check that an image exists
    if year_image is None:
        print("No image found for this year/region. Skipping.")
        continue

    year_image = year_image.toFloat()

    # Optional: keep all 64 bands (A00..A63). If you need fewer, select them:
    # year_image = year_image.select(['A00','A01','A02'])

    # Start Drive export (direct download of 64 bands over large areas will usually fail)
    desc = f'{file_prefix}_embeddings_{year}'
    print("  Exporting to Google Drive...")
    task = ee.batch.Export.image.toDrive(
        image=year_image,
        description=desc,
        folder=folder_name,
        fileNamePrefix=f'{file_prefix}_{year}_{export_scale_m}m',
        region=region,
        scale=export_scale_m,
        crs=export_crs,
        maxPixels=1_000_000_000_000
    )
    task.start()
    downloaded_tasks.append(task.id)
    print(f"  ✓ Export task started: {task.id}")

print("\nAll export tasks submitted.")
print("Check status in the Code Editor Tasks tab or at https://code.earthengine.google.com/tasks")

Step 4: Downloading embeddings for each year...

YEAR 2017
  Exporting to Google Drive...
  ✓ Export task started: OY76PM4TX5K4OBDIXQ4BABFK
YEAR 2018
  Exporting to Google Drive...
  ✓ Export task started: 4HJXKYZP7UPPO4DTYZYOYRXN
YEAR 2019
  Exporting to Google Drive...
  ✓ Export task started: FQU7T5LRAX2VBKMKOKRY2QY7
YEAR 2020
  Exporting to Google Drive...
  ✓ Export task started: 6QBLPGEKEYYQAUOXQTWYN22Z
YEAR 2021
  Exporting to Google Drive...
  ✓ Export task started: UIHCJWEXH3GMKQPCXR2FJDDQ
YEAR 2022
  Exporting to Google Drive...
  ✓ Export task started: ZJFC3F7NRKQXYWGMRLH6GZ2D
YEAR 2023
  Exporting to Google Drive...
  ✓ Export task started: 6Q62DOYEG44GYFHQQXBW47YQ
YEAR 2024
  Exporting to Google Drive...
  ✓ Export task started: 4QAZNIA53I2N3VN55GI2STJH

All export tasks submitted.
Check status in the Code Editor Tasks tab or at https://code.earthengine.google.com/tasks
